In [98]:
import os
import fitz  # PyMuPDF pour extraire le texte des PDF
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [99]:
# Télécharger les stopwords en français
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
# Fonction pour extraire le texte d'un PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = " ".join([page.get_text("text") for page in doc])
        return text
    except Exception as e:
        print(f"Erreur de lecture {pdf_path}: {e}")
        return ""

In [101]:
# Fonction de prétraitement du texte
def preprocess_text(text):
    text = unidecode(text)  # Supprime les accents (ex: "développeur" -> "developpeur")
    text = re.sub(r'\s+', ' ', text)  # Supprime les espaces multiples
    text = re.sub(r'[^\w\s]', '', text)  # Supprime la ponctuation
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in stop_words])
    return text


In [102]:
# Dossier contenant les CVs (modifier selon votre chemin)
cv_folder = r"C:\Users\Asus\Desktop\donner\cv"

In [103]:
# Description du poste en français
job_description = """
Nous recherchons un ingénieur en informatique spécialisé en Python, Intelligence Artificielle et Développement Web.
Expérience avec Flask, Angular et bases de données est un plus.
"""  # Modifier selon le besoin

In [104]:
# Lire et traiter les CVs
cv_files = [os.path.join(cv_folder, f) for f in os.listdir(cv_folder) if f.endswith('.pdf')]
cv_texts = [preprocess_text(extract_text_from_pdf(f)) for f in cv_files]


In [105]:
# Ajouter la description du poste comme référence
documents = [preprocess_text(job_description)] + cv_texts

# Transformer les textes en vecteurs TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

In [106]:
# Calculer la similarité cosinus entre la description du poste et les CVs
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Classer les CVs en fonction de leur pertinence
ranked_indices = np.argsort(cosine_similarities)[::-1]  # Tri décroissant


In [107]:
# Affichage des résultats
print("\n🔹 Classement des CVs selon la pertinence:")
for i, idx in enumerate(ranked_indices):
    print(f"Rang {i+1}: {cv_files[idx]} - Score: {cosine_similarities[idx]:.4f}")


🔹 Classement des CVs selon la pertinence:
Rang 1: C:\Users\Asus\Desktop\donner\cv\my cv.pdf - Score: 0.1053
Rang 2: C:\Users\Asus\Desktop\donner\cv\cv7.pdf - Score: 0.0799
Rang 3: C:\Users\Asus\Desktop\donner\cv\CV Ghaya.pdf.pdf - Score: 0.0729
Rang 4: C:\Users\Asus\Desktop\donner\cv\cv aziz.pdf - Score: 0.0696
Rang 5: C:\Users\Asus\Desktop\donner\cv\cv emma2.pdf - Score: 0.0624
Rang 6: C:\Users\Asus\Desktop\donner\cv\yassine+CV (1).pdf - Score: 0.0472
Rang 7: C:\Users\Asus\Desktop\donner\cv\cv emma1.pdf - Score: 0.0000
